## Supply Chain Optimisation using Decision Trees with Genetic Algorithms

In [ ]:
import numpy as np
from GDTR import GeneticDecisionTreeRegressor
from or_gym.envs.supply_chain.inventory_management import InvManagementMasterEnv
import random
import json
import os
from pathlib import Path
import pickle
%load_ext autoreload
%autoreload 2

### Initialise environment

In [ ]:
env = InvManagementMasterEnv(periods=30, c=[4, 3, 2], I0=[3, 2, 2], dist_param={'mu': 1.2}, L=[1, 1, 1])

action_space = env.action_space
upper_bounds = []
lower_bounds = []

# create bounds for inputs
for i in range(action_space.shape[0]): # action_space.shape[0] gives us the number of stages
    upper_bounds.append(action_space.high[i])
    lower_bounds.append(action_space.low[i])

In [ ]:
def generate_base_stock_action(env, state, z):
        '''
        Sample action (number of units to request) based on a base-stock policy (order up to z policy)
        z = [integer list; dimension |Stages| - 1] base stock level (no inventory at the last stage)
        '''
        c = env.supply_capacity
        m = env.num_stages
        IP = state # extract inventory position
        
        try:
            dimz = len(z)
        except:
            dimz = 1
        assert dimz == m-1, "Wrong dimension on base stock level vector. Should be # Stages - 1."
        
        #print(type(IP))
        #print(IP)
        # calculate total inventory position at the beginning of period n
        R = z - IP # replenishmet order to reach zopt

        Im = np.append(IP[1:], np.Inf)
        # check if R can actually be fulfilled (capacity and inventory constraints: NOTE: assume that inventory is at 5*capacity)
        Rpos = np.column_stack((np.zeros(len(R)),R)) # augmented materix to get replenishment only if positive
        A = np.column_stack((c, np.max(Rpos,axis=1), Im)) # augmented matrix with c, R, and I_m+1 as columns

        R = np.min(A, axis = 1)
        
        return R

In [ ]:
def standardise_inputs(inputs):
    mean_vals = np.mean(inputs, axis=0)
    std_dev_vals = np.std(inputs, axis=0)
    
    standardized_inputs = (inputs - mean_vals) / std_dev_vals
    
    return standardized_inputs, mean_vals, std_dev_vals


### GA loop, half-heuristic half-random initialisation

In [ ]:

num_init_trees = 1320
num_random_trees = 660 # = (6 + 5*4*3) * 20
num_heuristic_trees = 660

# params to investigate
depth_list = [3, 5, 10, 13]
gen_split = [[50, 30, 20], [40, 40, 20], [30, 40, 30], [20, 50, 30], [10, 50, 40]]


for d in depth_list:
    print(f'currently at {d} depth')
    for g in gen_split:
        print(f'currently at {g} combination')
        GDTR_list = []
        mean_arr = np.array([])
        std_dev_arr = np.array([])
        
        for i in range(num_random_trees):
            tree = GeneticDecisionTreeRegressor(max_depth=d, n_outputs=3, mutation_percentage=50)
            random_seed = i
            
            # --------- generate random actions ---------
            # -------------------------------------------
            # specify number of datapoints on which each DT should be trained
            training_data_num = 10

            # generate sample action data for each of the stages
            stage_0 = np.array([random.randint(lower_bounds[0], upper_bounds[0]) for i in range(training_data_num)])
            stage_1 = np.array([random.randint(lower_bounds[1], upper_bounds[1]) for i in range(training_data_num)])
            stage_2 = np.array([random.randint(lower_bounds[2], upper_bounds[2]) for i in range(training_data_num)])

            # combine into one array, where each element contains the actions for all three stages at a timestep
            stages = np.column_stack([stage_0, stage_1, stage_2])
            
            
            # --------- generate random inventory levels ---------
            # ----------------------------------------------------
            # generate random invetory levels (needed for definig different states)
            inv_0 = np.array([random.randint(lower_bounds[0], upper_bounds[0]*5) for i in range(training_data_num)])
            inv_1 = np.array([random.randint(lower_bounds[1], upper_bounds[0]*5) for i in range(training_data_num)])
            inv_2 = np.array([random.randint(lower_bounds[2], upper_bounds[0]*5) for i in range(training_data_num)])

            inv_levels = np.column_stack([inv_0, inv_1, inv_2])
            
            
            # --------- combine the two into a random state ---------
            # -------------------------------------------------------
            states = np.hstack([inv_levels, stages])
            
            std_states, means, std_devs = standardise_inputs(states)
            
            try:
                mean_arr = np.vstack([mean_arr, [means]])
                std_dev_arr = np.vstack([std_dev_arr, [std_devs]])
            except ValueError:
                mean_arr = np.array(means)
                std_dev_arr = np.array(std_devs)

    
            # --------- generate random actions again ---------
            # -------------------------------------------------
            # generate sample action data for each of the stages
            stage_0a = np.array([random.randint(lower_bounds[0], upper_bounds[0]) for i in range(training_data_num)])
            stage_1a = np.array([random.randint(lower_bounds[1], upper_bounds[1]) for i in range(training_data_num)])
            stage_2a = np.array([random.randint(lower_bounds[2], upper_bounds[2]) for i in range(training_data_num)])

            # combine into one array, where each element contains the actions for all three stages at a timestep
            actions = np.column_stack([stage_0a, stage_1a, stage_2a])
            
            
            # --------- train the DT on the training data [.fit()] ---------
            # --------------------------------------------------------------
            tree.fit(std_states, actions)
            
            
            # --------- add the DT to our dictionary of DTs ---------
            # -------------------------------------------------------
            GDTR_list.append(tree)
        
        
        for i in range(num_heuristic_trees):
            # initialise trees based on base-stock policy; also add stochasticity
            h_tree = GeneticDecisionTreeRegressor(max_depth=d, n_outputs=3) 
            h_random_seed = i
            h_training_data_num = 200
            z = [7, 6, 5] # NOTE: this is the optimal base stock level, as found during benchmarking the base-stock policy separately
            
            # --------- generate random actions to describe the state  ---------
            # ------------------------------------------------------------------
            
            # generate sample action data for each of the stages (to be used as part of the state)
            h_stage_0 = np.array([random.randint(lower_bounds[0], upper_bounds[0]) for i in range(h_training_data_num)])
            h_stage_1 = np.array([random.randint(lower_bounds[1], upper_bounds[1]) for i in range(h_training_data_num)])
            h_stage_2 = np.array([random.randint(lower_bounds[2], upper_bounds[2]) for i in range(h_training_data_num)])

            # combine into one array, where each element contains the actions for all three stages at a timestep
            h_stages = np.column_stack([h_stage_0, h_stage_1, h_stage_2])
            
            # --------- generate random inventory levels ---------
            # ----------------------------------------------------
            # generate random invetory levels (needed for definig different states)
            h_inv_0 = np.array([random.randint(lower_bounds[0], upper_bounds[0]*5) for i in range(h_training_data_num)])
            h_inv_1 = np.array([random.randint(lower_bounds[1], upper_bounds[0]*5) for i in range(h_training_data_num)])
            h_inv_2 = np.array([random.randint(lower_bounds[2], upper_bounds[0]*5) for i in range(h_training_data_num)])

            h_inv_levels = np.column_stack([h_inv_0, h_inv_1, h_inv_2])
            
            
            # --------- combine the two into a random state ---------
            # -------------------------------------------------------
            h_states = np.hstack([h_inv_levels, h_stages])
            #print(f'len stages: {len(h_stages)} \nlen inv levels: {len(h_inv_levels)}')
            
            h_std_states, means, std_devs = standardise_inputs(h_states)
            
            try:
                mean_arr = np.vstack([mean_arr, [means]])
                std_dev_arr = np.vstack([std_dev_arr, [std_devs]])
            except ValueError:
                mean_arr = np.array(means)
                std_dev_arr = np.array(std_devs)

            
            # --------- generate base-stock action based on the states ---------
            # ------------------------------------------------------------------
            excpetion_counter = 0
            for h_inv_lvl in h_inv_levels:
                h_action = generate_base_stock_action(env, h_inv_lvl, z)
                try:
                    h_actions = np.vstack((h_actions, h_action))
                except:
                    h_actions = h_action
                    excpetion_counter += 1
            #print(f'exceptions: {excpetion_counter}')
            
            #print(f'len std_states: {len(h_std_states)} \nlen actions: {len(h_actions)}. shape actions: {h_actions.shape} \nlen states: {len(h_std_states)}. shape states: {h_states.shape}')
            ## train tree on generated 'optimal' state-action data
            h_tree.fit(h_std_states, h_actions)
            
            # add tree to gen 0 list
            GDTR_list.append(h_tree)
            del h_actions
            ########


        num_of_gens = 10 
        plotting_list = []
        avg_mean = np.average(mean_arr, axis=0)
        avg_std_dev = np.average(std_dev_arr, axis=0)

        for gen in range(num_of_gens):
            # --------- evalute trees' policies ---------
            # -------------------------------------------
            N_episodes = 10
            avg_total_rewards = {}

            for tree in GDTR_list:
                total_reward_dummy = np.array([])
                
                for i in range(N_episodes):
                    obs = env.reset()
                    done = False
                    
                    while not done:
                        std_obs = (obs - avg_mean) / avg_std_dev
                        action = tree.predict_action(std_obs)

                        # Take the defined action (place an order), and advance to the next time period by taking a "step"
                        obs, reward, done, _ = env.step(action)
                        total_reward_dummy = np.append(total_reward_dummy, [reward])
                        
                avg_total_rewards[tree] = np.average(total_reward_dummy)
    
    
            # --------- select only best performing 50% (and update GDTR_dict) ---------
            # --------------------------------------------------------------------------
            sorted_dict = sorted(avg_total_rewards.items(), key=lambda x: x[1], reverse=True) # sort the dictionary items based on values in descending order
            plotting_list.append(sorted_dict)
            
            # Calculate the index to slice the top x%
            index_to_slice = int(len(sorted_dict) * g[0]/100)

            # Get the keys (trees) corresponding to the top x% of values
            top_trees = [key for key, value in sorted_dict[:index_to_slice]]
            

            # --------- introduce crossovers to get an extra y% (and add to GDTR_dict) ---------
            # ----------------------------------------------------------------------------------- 
            trees_to_cross = random.choices(top_trees, k=round(num_init_trees*g[1]/100)*2)
                    
            for i in range(0, len(trees_to_cross), 2):
                first_tree = trees_to_cross[i]
                second_tree = trees_to_cross[i+1]
                
                crossed_tree = first_tree.generate_crossover(second_tree)
                
                top_trees.append(crossed_tree)
                    
                
            # --------- introduce mutations to get an extra z% (and add to GDTR_dict) ---------
            # ----------------------------------------------------------------------------------
            trees_to_mutate = random.choices(top_trees, k=round(num_init_trees*g[2]/100))
            
            for i in range(len(trees_to_mutate)):
                tree_to_mutate = trees_to_mutate[i]
                mutated_tree = tree_to_mutate.generate_mutation(len(obs))
                
                top_trees.append(mutated_tree)
            
            print(f'done with generation {gen}')
    
            # --------- update tree list, and go to the next generation ---------
            # -------------------------------------------------------------------
            GDTR_list = top_trees

In [ ]:
# --------- select overall best tree (and policy) ---------
# --------------------------------------------------------- 
N_episodes = 50
avg_total_rewards = {}

for tree in GDTR_list:
    total_reward_dummy = np.array([])
    
    for i in range(N_episodes):
        obs = env.reset()
        done = False
        
        while not done:
            std_obs = (obs - avg_mean) / avg_std_dev
            action = tree.predict_action(std_obs)

            obs, reward, done, _ = env.step(action)
            total_reward_dummy = np.append(total_reward_dummy, [reward])
            
    avg_total_rewards[tree] = np.average(total_reward_dummy)


sorted_dict = sorted(avg_total_rewards.items(), key=lambda x: x[1], reverse=True)
best_trees = sorted_dict[:5]

for tree in best_trees:
    total_reward_dummy = np.array([])
    
    for i in range(N_episodes):
        obs = env.reset()
        done = False
        
        while not done:
            std_obs = (obs - avg_mean) / avg_std_dev
            action = tree.predict_action(std_obs)

            obs, reward, done, _ = env.step(action)
            total_reward_dummy = np.append(total_reward_dummy, [reward])
            
    avg_total_rewards[tree] = np.average(total_reward_dummy)

sorted_dict = sorted(avg_total_rewards.items(), key=lambda x: x[1], reverse=True)
best_trees = sorted_dict[:5]